# Import Example Data Sets

In [ ]:
import sys
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = [16 / 1.5, 10 / 1.5]   # inch / cm = 2.54
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'
# plt.rcParams['savefig.frameon'] = False
from math import *

The library **sampledata** provides a number of datasets as pandas dataframes. 

In [ ]:
import exampledata

In [ ]:
from svmodels import SV, hole_New
from correlation_models import cvmodel

In [ ]:
%load_ext autoreload
%autoreload 2


## Periodic Sine Curves with Noise

sine_data() provides a sine curve with given period and amplitude, distorted with random noise (with given distribution). 

We get two dataframes - a complete one to plot the true population, and 25 random samples representing noisy measurements:

In [ ]:
df_all = exampledata.sine_data()
df_z = exampledata.sine_data(n_sample=25, noise_std = 1)

plt.plot(df_z.x, df_z.measurement, '.', label='noisy sample', color = "red")
plt.plot(df_all.x, df_all.truez, label="true value")
plt.errorbar(df_z.x, df_z.measurement, df_z.confidence95, fmt='.',
             label='confidence intervall', color="grey")

plt.legend()
plt.show()

In [ ]:
df_z

a number of parameters can be used to obtain different datasets, see below (here, the sample is taken afterwards for better code readability).

In [ ]:
df_all = exampledata.sine_data(start = 0,  stop = 800,  # intervall for predictor
              resolution = 500,   # number of samples to create in interval
              period = 500, amplitude = 10,  # shape of sine curve 
              noise_type="gaussian",   # type of noise
              noise_mean = 0,  # mean of noise
              noise_std = 0.5)  # standard deviation of noise
df_z = df_all.sample(50)

plt.plot(df_z.x, df_z.measurement, '.', label='noisy sample', color = "red")
plt.plot(df_all.x, df_all.truez, label="true value")
plt.errorbar(df_z.x, df_z.measurement, df_z.confidence95, fmt='.',
             label='confidence intervall', color="grey")

df_z.to_excel(r".\example_data\Periodic Sine\example_5.xlsx")

plt.legend()
plt.show()

In [ ]:
df_z

## Storing and Modifying Data

The dataframe can be saved (see cell above), and later be loaded again.
The following table ist a concanenated dataset for Example 2, with measurements of different confidence intervals.

In [ ]:
df_z = pd.read_excel(r".\example_data\Periodic Sine\example_2_modified.xlsx")

plt.plot(df_z.x, df_z.measurement, '.', label='noisy sample', color = "red")
plt.plot(df_all.x, df_all.truez, label="true value")
plt.errorbar(df_z.x, df_z.measurement, df_z.confidence95, fmt='.',
             label='confidence intervall', color="grey")

plt.legend()
plt.show()

In [ ]:
df_z

In [ ]:
P = np.array(df_z.dropna()[['x','y','truez', 'noise']])
P

In [ ]:

from scipy.spatial.distance import squareform, pdist

In [ ]:
# bandwidth, plus or minus bw meters
bw = 25
# lags in 25 days increments from zero to 2000
hs = np.arange(0, 4000, bw)
sv = SV( P, hs, bw )
plt.plot( sv[0], sv[1], '.-' )
#plt.errorbar(df_z.x, df_z.truez, df_z.noise, fmt='.',
             #label='confidence intervall', color="grey")
plt.xlabel('Lag [m]')
plt.ylabel('Semivariance')
plt.title('Sample Semivariogram') ;
#savefig('sample_semivariogram.png',fmt='png',dpi=200)

In [ ]:
varmodel = hole_New
C0=100.0
sp = cvmodel( P, model=varmodel, hs=np.arange(0,4000,25), bw=bw, Cn=0.0, svrange=250/(2*pi), C0=C0)
#C0 = C( P, hs[0], bw )

plt.plot( sv[0], sv[1], '.-' )
plt.plot( sv[0], C0 - sp( sv[0] ) ) ;
plt.title('Hole_N Model')
plt.ylabel('Semivariance')
plt.xlabel('Lag [m]')
#savefig('semivariogram_model_hole_N_highamplitude2.png',fmt='png',dpi=300)

In [ ]:
def krige( P, covfct, u, N ):
    '''
    Input  (P)     ndarray, data
           (covfct) modeling function
                    - spherical
                    - exponential
                    - gaussian
           (u)     unsampled point
           (N)     number of neighboring
                   points to consider
    '''

    assert N < len(P) + 1, "Number of Neighbors greater than number of data points"

    # mean of the variable
    mu = np.mean( P[:,2] )
 
    # distance between u and each data point in P
    d = np.sqrt( ( P[:,0]-u[0] )**2.0 + ( P[:,1]-u[1] )**2.0 )
    # add these distances to P
    P = np.vstack(( P.T, d )).T
    # sort P by these distances
    # take the first N of them
    P = P[d.argsort()[:N]]
 
    # apply the covariance model to the distances
    k = covfct( P[:,3] )
    # cast as a matrix
    k = np.matrix( k ).T
 
    # form a matrix of distances between existing data points
    K = squareform( pdist( P[:,:2] ) )
    # apply the covariance model to these distances
    K = covfct( K.ravel() )
    # re-cast as a NumPy array -- thanks M.L.
    K = np.array( K )
    # reshape into an array
    K = K.reshape(N,N)
    # cast as a matrix
    K = np.matrix( K )
    K = np.identity(7)*0.2+K
 
    # calculate the kriging weights
    weights = np.linalg.inv( K ) * k
    weights = np.array( weights )
 
    # calculate the residuals
    residuals = P[:,2] - mu
 
    # calculate the estimation
    estimation = np.dot( weights.T, residuals ) + mu
 
    return (float( estimation ), k, K, weights)

In [ ]:
# sampling intervall:
X0, X1 = 0., 1000.

# number of samples
n = 600
dx = (X1-X0)/n
print("Sampling resolution: {:.2f} days".format(dx))

# number of neighbors:
nn = 7
u = (400, 0)  # (x,y) - coordinates of unsampled points


In [ ]:
Z = np.zeros(n)
V = np.zeros(n)
for i in range(n):
    h, k, K, weights = krige( P, sp, (dx*i, 0), nn )
    Z[i] = h
    v = C0 - np.dot(k.T, weights)
    V[i] = max (v, 0)

In [ ]:
plt.rcParams["figure.figsize"] = (16, 12)
plt.fill_between([i*dx for i in range(n)], Z-3*np.sqrt(V), Z+3*np.sqrt(V), color='green', alpha=0.1, label="+/- 3 stdev")
plt.fill_between([i*dx for i in range(n)], Z-2*np.sqrt(V), Z+2*np.sqrt(V), color='green', alpha=0.2, label="+/- 2 stdev")
plt.fill_between([i*dx for i in range(n)], Z-1*np.sqrt(V), Z+1*np.sqrt(V), color='green', alpha=0.3, label="+/- 1 stdev")
plt.plot([i*dx for i in range(n)], Z, "g-", label="estimator")
plt.plot(df_all.x, df_all.truez, label="true value")

plt.legend(loc='upper right')
plt.ylabel('Water Level [m]')
plt.xlabel('Time [d]')

x1,x2,y1,y2 = plt.axis()

plt.axis((x1,x2,-20, 20))
#plt.savefig("result_hole_N_model_highAmplitude_SKwithtimeseries40nn.png")
plt.show()